In [4]:
import json
from pathlib import Path
import napari
import numpy as np
from tifffile import imread, imwrite


In [7]:
#############################################################################################
# VARIABLES  - You can change these variables to adapt the code to your dataset
BATCH = "2_annotators_CD4" # Batch name for the dataset - Should match the folder structure
MARKER = "CD4" # Marker name to be used in the dataset
#############################################################################################

In [8]:
# Constants
MARKER_DIR = Path("../data/" + BATCH + "/tiles_marker")
DAPI_DIR = Path("../data/" + BATCH + "/tiles_DAPI")
ANNOT_DIR = Path("../annotations/" + BATCH + "/")
ANNOT_DIR.mkdir(exist_ok=True)

In [9]:
# Prepare list of marker tiles
all_marker_tiles = sorted(MARKER_DIR.glob("*_" + MARKER + ".tiff"))

# find next un-annotated tile
def find_next_tile():
    for tile in all_marker_tiles:
        annot_path = ANNOT_DIR / (tile.stem + ".json")
        if not annot_path.exists():
            return tile
    return None

In [10]:
# Global viewer and state (will be set in next_tile())
viewer = napari.Viewer()
current = {}

In [11]:
def load_tile(tile: Path):
    viewer.layers.clear()
    dapi = DAPI_DIR / tile.name.replace("_" + MARKER + ".tiff", "_DAPI.tiff")
    if not dapi.exists():
        viewer.window.status = f"No matching DAPI for {tile}"
        return False
    img_marker = imread(tile)
    img_dapi = imread(dapi)
    viewer.add_image(img_dapi, name="DAPI", colormap="bop blue", opacity=0.5)
    viewer.add_image(img_marker, name=MARKER, colormap="gray", blending='additive')
    return True


In [14]:
@viewer.bind_key('Shift-B', overwrite=True)
def annotate_and_next(viewer):
    marker_layer = viewer.layers[MARKER] if MARKER in viewer.layers else None
    if marker_layer is None:
        viewer.window.status = "No marker layer loaded."
        return

    # Get contrast limits
    min_val, max_val = marker_layer.contrast_limits
    current_tile = current.get('marker')
    if current_tile is None:
        viewer.window.status = "No tile loaded."
        return

    # Save annotation
    annotation = {
        'min': float(min_val),
        'max': float(max_val),
        'tile': current_tile.name
    }
    save_path = ANNOT_DIR / (current_tile.stem + '.json')
    with open(save_path, 'w') as f:
        json.dump(annotation, f)

    viewer.window.status = f"Saved annotation for {current_tile.name}"

    # Remove previous layers
    for name in [MARKER, 'DAPI']:
        if name in viewer.layers:
            viewer.layers.remove(viewer.layers[name])

    # Load next tile
    next_tile = find_next_tile()
    if next_tile:
        current['marker'] = next_tile
        load_tile(next_tile)
        viewer.window.status = f"Loaded {next_tile.name}"
    else:
        viewer.window.status = "No tiles left to annotate — you're done!"


In [12]:
UPPER_RANGE = 1  # adjust if needed (e.g. 255 for 8-bit)

def adjust_contrast(min_shift=0, max_shift=0):
    try:
        layer = viewer.layers[MARKER]
        vmin, vmax = layer.contrast_limits
        current_range = vmax - vmin

        new_vmin = max(0, vmin + min_shift * current_range)
        new_vmax = min(UPPER_RANGE, vmax + max_shift * current_range)

        # Prevent vmin > vmax
        if new_vmin >= new_vmax:
            return

        layer.contrast_limits = (new_vmin, new_vmax)
        viewer.window.status = f"Range updated: ({int(new_vmin)} – {int(new_vmax)})"
    except KeyError:
        viewer.window.status = MARKER + " layer not loaded."

# Shift+A: decrease min
@viewer.bind_key('Shift-A', overwrite=True)
def decrease_min(viewer):
    adjust_contrast(min_shift=-0.1)

# Shift+S: increase min
@viewer.bind_key('Shift-S', overwrite=True)
def increase_min(viewer):
    adjust_contrast(min_shift=+0.1)

# Shift+Q: decrease max
@viewer.bind_key('Shift-Q', overwrite=True)
def decrease_max(viewer):
    adjust_contrast(max_shift=-0.1)

# Shift+W: increase max
@viewer.bind_key('Shift-W', overwrite=True)
def increase_max(viewer):
    adjust_contrast(max_shift=+0.1)

In [13]:
# Kick off with the first tile
first = find_next_tile()
if first:
    current['marker'] = first
    load_tile(first)
    viewer.window.status = f"Loaded {first}"
else:
    viewer.window.status = "No tiles to annotate — you're done!"